In [39]:
import time
import yaml
import numpy as np
import pandas as pd

from rich.console import Console
from rich.table import Table
from rich.progress import track
from rich.progress import Progress

from olympus.campaigns import Campaign

from atlas import __home__
from atlas.ot2.volume_converter import *



In [9]:
content = open(f'{__home__}/ot2/campaign_config.yaml', 'r')
campaign_config = yaml.full_load(content)
#print(campaign_config)

# compute molar target concentrations and total solvent volumes
campaign_config = compute_target_conc(campaign_config)

batch_size = campaign_config['general']['batch_size']

func_param_space, full_param_space, full_space_instructions = get_param_space(campaign_config)
value_space = get_value_space(campaign_config)
print('\nfunc param space')
print(func_param_space)
print('\nfull param space')
print(full_param_space)
print('\nfull space instructions')
print(full_space_instructions)
print('\nvalue space')
print(value_space)

# functional campaign
func_campaign = Campaign()
func_campaign.set_param_space(func_param_space)
func_campaign.set_value_space(value_space)

# full campaign
full_campaign = Campaign()
full_campaign.set_param_space(full_param_space)
full_campaign.set_value_space(value_space)


func param space
Discrete (name='paclitaxel', low=0.0, high=1.0, stride=0.1, options=[0.05, 0.1, 0.2, 0.4])
Discrete (name='cholesterol', low=0.0, high=1.0, stride=0.1, options=[0.4, 0.9])
Discrete (name='dmg-peg_2000', low=0.0, high=1.0, stride=0.1, options=[0.0, 0.015, 0.03, 0.05])

full param space
Discrete (name='paclitaxel', low=0.0, high=1.0, stride=0.1, options=[0.05, 0.1, 0.2, 0.4])
Discrete (name='cholesterol', low=0.0, high=1.0, stride=0.1, options=[0.4, 0.9])
Continuous (name='dspc', low=0.0, high=1.0, is_periodic=False)
Discrete (name='dmg-peg_2000', low=0.0, high=1.0, stride=0.1, options=[0.0, 0.015, 0.03, 0.05])

full space instructions
{'dspc': ['cholesterol', 'dmg-peg_2000']}

value space
Continuous (name='encapsulation_efficiency', low=0.0, high=1.0, is_periodic=False)


In [29]:
full_campaign.goal

'minimize'

In [32]:
table = Table(title='Experiment Configuration Parameter Space')

table.add_column('Parameter Name', justify='center', style='cyan', no_wrap=True)
table.add_column('Type', justify='center', style='cyan', no_wrap=True)
table.add_column('Range (# Options)', style='magenta', justify='center')

for param in full_param_space:
    if param.type=='continuous':
        range_ = f'[{param.low} - {param.high}]'
        num_options='N/A'
    else:
        range_ = 'N/A'
        num_options=len(param.options)
    table.add_row(
        param.name,
        param.type,
        f'{range_} ({num_options})',
        
    )
console = Console()
console.print(table)

     Experiment Configuration Parameter Space      
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Parameter Name ┃    Type    ┃ Range (# Options) ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│   paclitaxel   │  discrete  │      N/A (4)      │
│  cholesterol   │  discrete  │      N/A (2)      │
│      dspc      │ continuous │ [0.0 - 1.0] (N/A) │
│  dmg-peg_2000  │  discrete  │      N/A (4)      │
└────────────────┴────────────┴───────────────────┘

     Experiment Configuration Parameter Space      
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Parameter Name ┃    Type    ┃ Range (# Options) ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│   paclitaxel   │  discrete  │      N/A (4)      │
│  cholesterol   │  discrete  │      N/A (2)      │
│      dspc      │ continuous │ [0.0 - 1.0] (N/A) │
│  dmg-peg_2000  │  discrete  │      N/A (4)      │
└────────────────┴────────────┴───────────────────┘

In [33]:
table = Table(title='Experiment Configuration - Objective Space')

table.add_column('Objective Name', justify='center', style='cyan', no_wrap=True)
table.add_column('Type', justify='center', style='cyan', no_wrap=True)
table.add_column('Goal', style='magenta', justify='center')

for ix, value in enumerate(value_space):
    if len(value_space)==1:
        goal=full_campaign.goal
    else:
        goal=full_campaign.goal[ix]
    table.add_row(
        value.name,
        value.type,
        goal
    )

console = Console()
console.print(table)

     Experiment Configuration - Objective Space     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┓
┃      Objective Name      ┃    Type    ┃   Goal   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━┩
│ encapsulation_efficiency │ continuous │ minimize │
└──────────────────────────┴────────────┴──────────┘

In [38]:
for i in track(range(20), description="Processing..."):
    time.sleep(1)  # Simulate work being done

Processing... ━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━  55% 0:00:10

KeyboardInterrupt: 

In [40]:
with Progress() as progress:

    task1 = progress.add_task("[red]Downloading...", total=1000)
    task2 = progress.add_task("[green]Processing...", total=1000)
    task3 = progress.add_task("[cyan]Cooking...", total=1000)

    while not progress.finished:
        progress.update(task1, advance=0.5)
        progress.update(task2, advance=0.3)
        progress.update(task3, advance=0.9)
        time.sleep(0.02)


Downloading... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺  98% 0:00:01
Processing...  ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━  59% 0:00:28
Cooking...     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

KeyboardInterrupt: 